# Пользовательские ограничения для валидации данных

В этом ноутбуке мы рассмотрим создание и использование пользовательских ограничений для валидации данных.

## Подготовка окружения

In [ ]:
import sys
sys.path.append('../../')

import pandas as pd
import numpy as np
from core.data.schema import DataSchema, ColumnSchema, DataType
from core.data.validation import SchemaValidator

## Создание схемы с пользовательскими ограничениями

In [ ]:
# Создание схемы данных с пользовательскими ограничениями
schema = DataSchema([
    ColumnSchema(
        name='id',
        data_type=DataType.INTEGER,
        required=True,
        constraints={
            'unique': True,
            'positive': True  # Пользовательское ограничение
        }
    ),
    ColumnSchema(
        name='name',
        data_type=DataType.STRING,
        required=True,
        constraints={
            'min_length': 2,  # Пользовательское ограничение
            'max_length': 50  # Пользовательское ограничение
        }
    ),
    ColumnSchema(
        name='age',
        data_type=DataType.INTEGER,
        required=True,
        constraints={
            'min': 0,
            'max': 150,
            'even': True  # Пользовательское ограничение
        }
    ),
    ColumnSchema(
        name='salary',
        data_type=DataType.FLOAT,
        required=True,
        constraints={
            'min': 0,
            'max': 1000000,
            'multiple_of': 1000  # Пользовательское ограничение
        }
    )
])

## Создание валидатора с пользовательскими проверками

In [ ]:
class CustomSchemaValidator(SchemaValidator):
    def _validate_integer(self, series: pd.Series, schema: ColumnSchema) -> None:
        super()._validate_integer(series, schema)
        
        # Проверка на положительные числа
        if schema.constraints.get('positive', False):
            if (series <= 0).any():
                raise ValueError(f"Колонка {schema.name} должна содержать только положительные числа")
        
        # Проверка на четные числа
        if schema.constraints.get('even', False):
            if (series % 2 != 0).any():
                raise ValueError(f"Колонка {schema.name} должна содержать только четные числа")

    def _validate_string(self, series: pd.Series, schema: ColumnSchema) -> None:
        super()._validate_string(series, schema)
        
        # Проверка длины строки
        min_length = schema.constraints.get('min_length')
        max_length = schema.constraints.get('max_length')
        
        if min_length is not None:
            if (series.str.len() < min_length).any():
                raise ValueError(f"Колонка {schema.name} должна содержать строки длиной не менее {min_length} символов")
        
        if max_length is not None:
            if (series.str.len() > max_length).any():
                raise ValueError(f"Колонка {schema.name} должна содержать строки длиной не более {max_length} символов")

    def _validate_float(self, series: pd.Series, schema: ColumnSchema) -> None:
        super()._validate_float(series, schema)
        
        # Проверка на кратность
        multiple_of = schema.constraints.get('multiple_of')
        if multiple_of is not None:
            if (series % multiple_of != 0).any():
                raise ValueError(f"Колонка {schema.name} должна содержать числа, кратные {multiple_of}")

## Подготовка тестовых данных

In [ ]:
# Создание корректных данных
valid_data = pd.DataFrame({
    'id': [1, 2, 3],
    'name': ['John', 'Jane', 'Bob'],
    'age': [20, 30, 40],
    'salary': [1000, 2000, 3000]
})

# Создание некорректных данных
invalid_data = pd.DataFrame({
    'id': [-1, 2, 3],  # Отрицательный id
    'name': ['J', 'Jane', 'Bob'],  # Слишком короткое имя
    'age': [21, 30, 41],  # Нечетные числа
    'salary': [1500, 2000, 3500]  # Не кратны 1000
})

## Создание и использование валидатора

In [ ]:
# Создание валидатора
validator = CustomSchemaValidator(schema)

# Валидация корректных данных
try:
    validator.validate(valid_data)
    print("Данные успешно валидированы")
except ValueError as e:
    print(f"Ошибка валидации: {e}")

# Валидация некорректных данных
try:
    validator.validate(invalid_data)
except ValueError as e:
    print(f"Ошибка валидации: {e}")

## Валидация отдельных ограничений

In [ ]:
# Проверка ограничения на положительные числа
try:
    validator._validate_integer(invalid_data['id'], schema.get_column_info('id'))
except ValueError as e:
    print(f"Ошибка валидации id: {e}")

# Проверка ограничения на длину строки
try:
    validator._validate_string(invalid_data['name'], schema.get_column_info('name'))
except ValueError as e:
    print(f"Ошибка валидации имени: {e}")

# Проверка ограничения на четность
try:
    validator._validate_integer(invalid_data['age'], schema.get_column_info('age'))
except ValueError as e:
    print(f"Ошибка валидации возраста: {e}")

# Проверка ограничения на кратность
try:
    validator._validate_float(invalid_data['salary'], schema.get_column_info('salary'))
except ValueError as e:
    print(f"Ошибка валидации зарплаты: {e}")